<img src='https://repository-images.githubusercontent.com/121802384/c355bb80-7d42-11e9-9e0e-4729609f9fbc' alt='WRF-Hydro Logo' width="15%"/>

# Lesson 6 - Lake options for the IWAAs configuration

## Overview
In [Lesson 4](Lesson-4-run-options.ipynb) we covered some experiments in tinkering with terrain physics. In this lesson, we specifically experiment with lake physics options. 

We conduct two experiments in this lesson: 

1. Adding lakes to the simulation
2. Modifying lake parameters

See [here](https://ral.ucar.edu/sites/default/files/public/Channel%20Routing_Lakes.pdf) for reference material on lakes and channel routing in WRF-Hydro.

### Compiling WRF-Hydro
We do not need to modify the code binary to conduct lake physics experiments, so we will use the compiled code from the template directory.  

If you need to recompile the code, refer to the instructions in [Lesson 2](Lesson-2-run.ipynb).

### Creating the simulation directory and running a baseline (Lake off) WRF-Hydro simulation
Below are the commands from Lesson 2 to create the simulation directory and run a WRF-Hydro simulation. This simulation will be run exactly as was done in Lesson 2 and will serve as our baseline simulation. We will modify run-time options in subsequent simulations in the lesson and compare the outputs.  

In [ ]:
%%bash

# Make a new directory for our baseline simulation
mkdir -p ~/wrf-hydro-training/output/lesson6/run_IWAAs_baseline
cd ~/wrf-hydro-training/output/lesson6/run_IWAAs_baseline

# Copy our model files and domain files to the simulation directory
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/*.TBL .
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/wrf_hydro.exe .
cp -r ~/wrf-hydro-training/example_case/IWAAs/DOMAIN .

# Create symbolic links to forcing and restarts
ln -sf ~/wrf-hydro-training/example_case/FORCING .
ln -sf ~/wrf-hydro-training/example_case/IWAAs/RESTART .

# Copy namelist files
cp ~/wrf-hydro-training/example_case/IWAAs/namelist.hrldas .
cp ~/wrf-hydro-training/example_case/IWAAs/hydro.namelist .

**Step 3: First Specify the lake parameter file (Line #159 in `hydro.namelist` in this exercise):**

<pre>
! Specify the lake parameter file (e.g.: "LAKEPARM.nc" for netcdf or "LAKEPARM.TBL" for text).
! Note REQUIRED if lakes are on.
<span style="color:blue">!route_lake_f = "./DOMAIN/LAKEPARM.nc"</span>
</pre>

**Second, Turn on the CHRTOUT output files in the `hydro.namelist` file. We need the CHRTOUT files in the next steps.** 
<pre>
<span style="color:blue">CHRTOUT_DOMAIN = 1           ! Netcdf point timeseries output at all channel points (1d)</span>
<span style="color:blue">                             ! 0 = no output, 1 = output</span>
</pre>

In [ ]:
%%bash
# Run the simulation
cd ~/wrf-hydro-training/output/lesson6/run_IWAAs_baseline
mpirun -np 8 ./wrf_hydro.exe >> run.log 2>&1

In [ ]:
%%bash

# Check that it finished successfully
tail -1 ~/wrf-hydro-training/output/lesson6/run_IWAAs_baseline/diag_hydro.00000

## Setup for experiments

We will create a new simulation directory for each experiment. First, we will make a new simulation directory as we did with the baseline run above and use this as a template for creating multiple new simulation directories.

**Step 1: Create a simulation directory to use as a template**

In [ ]:
%%bash

# Make a new directory for our baseline simulation
mkdir -p ~/wrf-hydro-training/output/lesson6/run_IWAAs_template
cd ~/wrf-hydro-training/output/lesson6/run_IWAAs_template

# Copy our model files to the simulation directory
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/*.TBL .
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/wrf_hydro.exe .
cp -as $HOME/wrf-hydro-training/example_case/IWAAs/DOMAIN .

# Create symbolic links
ln -sf $HOME/wrf-hydro-training/example_case/FORCING .
ln -sf $HOME/wrf-hydro-training/example_case/IWAAs/RESTART .

# Copy namelist files
cp ~/wrf-hydro-training/example_case/IWAAs/namelist.hrldas .
cp ~/wrf-hydro-training/example_case/IWAAs/hydro.namelist .

**Step 2: View the contents**

In [ ]:
%%bash

ls ~/wrf-hydro-training/output/lesson6/run_IWAAs_template

### Review the baseline
First, we will recall the hydrograph from location below the lake, at the outlet of the basin: 

In [ ]:
# Load the required libraries
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

xr.set_options(display_style="html")
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
chanobs_baseline = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson6/run_IWAAs_baseline/*CHANOBS*',
                            combine='by_coords')
obs = pd.read_csv('/home/docker/wrf-hydro-training/example_case/USGS_obs.csv',dtype=str)
obs['dateTime'] = pd.to_datetime(obs['dateTime'])
obs['streamflow_cms'] = pd.to_numeric(obs['streamflow_cms'])

**Plot the default hydrograph at the main outlet of the basin**

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Hydrographs for lakes off at stream at the main outlet of the basin',fontsize=24)
chanobs_baseline.sel(feature_id = 23123539).streamflow.plot(label='Lakes off',
                                                        color='black',
                                                        linestyle='--')
obs[obs['site_no'] == '13010065'].plot(x='dateTime',
                                       y='streamflow_cms',
                                       ax=axes,
                                       label='Observed',
                                       color='grey')
plot_times = chanobs_baseline.time.data
plot_time_buffer = pd.to_timedelta(2, unit='D') 
plt.xlim([plot_times.min()-plot_time_buffer, plot_times.max()+plot_time_buffer])
plt.legend()
plt.show()

## Experiment 1 - Adding the lakes to the simulation

### Background
In the IWAAs channel routing configuration, WRF-Hydro can be run with lakes on or off. 

In this configuration, the lakes are represented as objects connected by channel reaches, rather than explicitly defined on the grid. 
The only change to execute with and without lakes are edits in the `hydro.namelist`. See below for details.

Note: In the grid-based routing, the lakes are explicitly represented on the fine grid (i.e. in the `LAKEGRID` variable in the `Fulldom_hires.nc` file). This is different from how lakes are represented in reach-based routing. Therefore, if a user wants to run the grid-based routing configuration without lakes, they should run the GIS pre-processing tools and NOT specify a lake polygon file. This will create a `Fulldom_hires.nc` file with the LAKEGRID variable = -9999 (fill values). 

### Objective
Run a simulation for the IWAAs configuration **with** representing lakes and compare the resulting hydrographs at a gage downstream of the lake. 

### Create experiment simulation directory

In [ ]:
%%bash

# Make a new directory for our baseline simulation
mkdir -p ~/wrf-hydro-training/output/lesson6/run_lakes_on
cd ~/wrf-hydro-training/output/lesson6/run_lakes_on
# Copy our model files to the simulation directory
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/*.TBL .
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/wrf_hydro.exe .
cp -as $HOME/wrf-hydro-training/example_case/IWAAs/DOMAIN .

# Create symbolic links
ln -sf $HOME/wrf-hydro-training/example_case/FORCING .
ln -sf $HOME/wrf-hydro-training/example_case/IWAAs/RESTART .

# Copy namelist files
cp ~/wrf-hydro-training/example_case/IWAAs/namelist.hrldas .
cp ~/wrf-hydro-training/example_case/IWAAs/hydro.namelist .

**Step 3: Check that lakes are on in the `hydro.namelist` file. There are two modifications:**

There are two options in the [hydro.namelist](../output/lesson6/run_IWAAs_template/hydro.namelist) to turn lakes on.

First specify the lake parameter file (Line #159 in hydro.namelist in this exercise):

<pre>
! Specify the lake parameter file (e.g.: "LAKEPARM.nc" for netcdf or "LAKEPARM.TBL" for text).
! Note REQUIRED if lakes are on.
<span style="color:blue">route_lake_f = "./DOMAIN/LAKEPARM.nc"</span>
</pre>

Second is turning on the `lakeout` and `CHRTOUT` output files.
<pre>
<span style="color:blue">outlake = 1                  ! Netcdf grid of lake values (1d)</span>
<span style="color:blue">                             ! 0 = no output, 1 = output</span>

<span style="color:blue">CHRTOUT_DOMAIN = 1           ! Netcdf point timeseries output at all channel points (1d)</span>
<span style="color:blue">                             ! 0 = no output, 1 = output</span>
<pre>

**Step 1: Run the simulations**

In [ ]:
%%bash

cd ~/wrf-hydro-training/output/lesson6/run_lakes_on
mpirun -np 8 ./wrf_hydro.exe >> run.log 2>&1

**Step 2: Check that the simulation finished successfully**

In [ ]:
%%bash

tail -1 ~/wrf-hydro-training/output/lesson6/run_lakes_on/diag_hydro.00000

### Results
We will now look at the differences in streamflow between our baseline run with lakes off and our experiment with lakes on.

We will use Python and the `xarray` library to load the data and plot hydrographs. For an intro to these tools, please see [Lesson 3](Lesson-3-visualize.ipynb).

**Load the CHRTOUT streamflow datasets**


In [ ]:
%%time
chrtout_baseline = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson6/run_IWAAs_baseline/*CHRTOUT*',
                            combine='by_coords')
chrtout_lakes_on = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson6/run_lakes_on/*CHRTOUT*',
                            combine='by_coords')
obs = pd.read_csv('/home/docker/wrf-hydro-training/example_case/USGS_obs.csv',dtype=str)
obs['dateTime'] = pd.to_datetime(obs['dateTime'])
obs['streamflow_cms'] = pd.to_numeric(obs['streamflow_cms'])

**Plot the hydrographs at the stream location upstream of the lake to ensure they are identical.**

In [ ]:
%%time
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Hydrographs for lakes on and off at stream location upstream of lake',fontsize=24)
chrtout_lakes_on.sel(feature_id = 23123871).streamflow.plot(label='Lakes on',
                                                        color='blue',
                                                        linestyle='-')
chrtout_baseline.sel(feature_id = 23123871).streamflow.plot(label='Lakes off',
                                                        color='black',
                                                        linestyle='--')
plt.legend()
plt.show()

### Discussion
The streamflow values at the stream location upstream of the lake were identical between the simulation with lakes represented and the simulation without lakes represented. This is expected.. and if we did not see this result, it would be an indication of an error somewhere in the process.  
However, at the stream location downstream of a lake we see that streamflow is considerably higher in the simulation without lakes represented. This reflects the expected behavior of the lake attenuating streamflow at the outlet due to storage. 

**Plot the hydrographs at the stream location downstream of the lake**

In [ ]:
%%time
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Hydrographs for lakes on and off at the main outlet of the basin',fontsize=24)
chrtout_lakes_on.sel(feature_id = 23123539).streamflow.plot(label='Lakes on',
                                                        color='blue',
                                                        linestyle='-')
chrtout_baseline.sel(feature_id = 23123539).streamflow.plot(label='Lakes off',
                                                        color='black',
                                                        linestyle='--')
obs[obs['site_no'] == '13010065'].plot(x='dateTime',
                                         y='streamflow_cms',
                                         ax=axes,
                                         label='Observed',
                                         color='grey')
plot_times = chanobs_baseline.time.data
plot_time_buffer = pd.to_timedelta(2, unit='D') 
plt.xlim([plot_times.min()-plot_time_buffer, plot_times.max()+plot_time_buffer])
plt.legend()
plt.show()

### Summary
The 'lakes-off' simulation shows a higher peak compared with the 'lakes-on'. We confirm that this lake serves to attenuate flow during a high event.

# Experiment 2 - Modifying lake parameters in `LAKEPARM.nc`

### Background
The orifice and weir lake parameters populated into LAKEPARM.nc by the GIS pre-processing tools are essentially placeholders for the user to replace with their own data. The accuracy of these parameters can have a significant impact on the storage and thus outflow of the lake. **We highly recommend replacing the default lake parameters with known values!**

### Objective
In this exercise, we will update one of these parameters, the weir length - `WeirL`, with an estimate from observations and explore the impact on the streamflow at the downstream gauge.

### First, let's make a new directory to conduct this experiment.

In [ ]:
%%bash

cp -r ~/wrf-hydro-training/output/lesson6/run_IWAAs_template \
~/wrf-hydro-training/output/lesson6/run_lakeparm_edit

### Before modifying anything, let's check the current default value of the WeirL in `LAKEPARM.nc`

In [ ]:
%%bash

ncdump -v WeirL ~/wrf-hydro-training/output/lesson6/run_lakeparm_edit/DOMAIN/LAKEPARM.nc

### Edit the lake parameter file and run the simulation
**Step 1: Edit the `LAKEPARM.nc` file**

NCO (NetCDF Operators, http://nco.sourceforge.net/) is a set of useful utilities to manipulate NetCDF files. We will use the NCO command `ncap2` to edit the WeirL variable.

In [ ]:
%%bash
ncap2 -s "where(lake_id==120053977) WeirL=12" ~/wrf-hydro-training/output/lesson6/run_lakeparm_edit/DOMAIN/LAKEPARM.nc ~/wrf-hydro-training/output/lesson6/run_lakeparm_edit/DOMAIN/LAKEPARM_WeirL_12.nc

**Step 2: Verify that the change was made**

We will check the value of the `WeirL` variable in our new `LAKEPARM_WeirL_12.nc` file against the original `LAKEPARM.nc` file.

In [ ]:
%%bash

ncdump -v WeirL ~/wrf-hydro-training/output/lesson6/run_lakeparm_edit/DOMAIN/LAKEPARM_WeirL_12.nc

**Step 3: Edit the *hydro.namelist* to use the new *LAKEPARM_WeirL_12.nc***

We will change the `route_lake_f` option in the [hydro.namelist](../output/lesson6/run_lakeparm_edit/hydro.namelist) to use the edited lakeparm file. 

First specify the lake parameter file (Line #159 in hydro.namelist in this excercise):

<pre>
! Specify the lake parameter file (e.g.: "LAKEPARM.nc" for netcdf or "LAKEPARM.TBL" for text).
! Note REQUIRED if lakes are on.
<span style="color:blue">route_lake_f = "./DOMAIN/LAKEPARM_WeirL_12.nc"</span>
</pre>

Second is turning on the `lakeout` output files.
<pre>
<span style="color:blue">outlake = 1                  ! Netcdf grid of lake values (1d)</span>
<span style="color:blue">                             ! 0 = no output, 1 = output</span>

<span style="color:blue">CHRTOUT_DOMAIN = 1           ! Netcdf point timeseries output at all channel points (1d)</span>
<span style="color:blue">                             ! 0 = no output, 1 = output</span>
<pre>

**Step 4: Run the simulation**

In [ ]:
%%bash

cd ~/wrf-hydro-training/output/lesson6/run_lakeparm_edit
mpirun -np 8 ./wrf_hydro.exe >> run.log 2>&1

In [ ]:
%%bash

tail -1 ~/wrf-hydro-training/output/lesson6/run_lakeparm_edit/diag_hydro.00000

### Results
We will now look at the differences in streamflow between our baseline (Lakes off) run with the original WeirL of 10m and our edited WeirL of 12m.

We will use Python and the `xarray` library to load the data and plot hydrographs. For an intro to these tools, please see [Lesson 3](Lesson-3-visualize.ipynb).

**Load the CHANOBS streamflow datasets**

We are going to use the *CHANOBS* files because it will limit the number of grid cells to only those which we have specified have a gage.

In [ ]:
chrtout_lakeparm_edit = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson6/run_lakeparm_edit/*CHRTOUT*',
                            combine='by_coords')
obs = pd.read_csv('/home/docker/wrf-hydro-training/example_case/USGS_obs.csv',dtype=str)
obs['dateTime'] = pd.to_datetime(obs['dateTime'])
obs['streamflow_cms'] = pd.to_numeric(obs['streamflow_cms'])

**Plot the hydrographs**

**Compare the result from experiment 1, where the lakes were added.**

In [ ]:
%%time
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Hydrographs of lake off, lake on, and modified weir length.',fontsize=24)
chrtout_lakeparm_edit.sel(feature_id = 23123539).streamflow.plot(label='WeirL = 12m',
                                                        color='blue',
                                                        linestyle='-')
chrtout_baseline.sel(feature_id = 23123539).streamflow.plot(label='Lake off',
                                                        color='black',
                                                        linestyle='--')
chrtout_lakes_on.sel(feature_id = 23123539).streamflow.plot(label='WeirL = 10m',
                                                        color='red',
                                                        linestyle='--')
obs[obs['site_no'] == '13010065'].plot(x='dateTime',
                                       y='streamflow_cms',
                                       ax=axes,
                                       label='Observed',
                                       color='grey')
plot_times = chanobs_baseline.time.data
plot_time_buffer = pd.to_timedelta(2, unit='D')
plt.xlim([plot_times.min()-plot_time_buffer, plot_times.max()+plot_time_buffer])
plt.legend()
plt.show()

### Summary
The increase in weir length causes more water to discharge from the lake earlier than the default.

**Load the LAKEOUT files to investigate three lake-related variables: inflow, outflow, and water surface elevation.**

In [ ]:
%%time
lakeout_lakes_on = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson6/run_lakes_on/*LAKEOUT*',
                            combine='by_coords')
lakeout_weir12 = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson6/run_lakeparm_edit/*LAKEOUT*',
                            combine='by_coords')

In [ ]:
%%time
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Outflow, lakes on (WeirL=10) and modified (WeirL=12m)',fontsize=24)

lakeout_weir12.sel(feature_id = 120053977).outflow.plot(label='WeirL = 12m',
                                                        color='blue',
                                                        linestyle='-')
lakeout_lakes_on.sel(feature_id = 120053977).outflow.plot(label='Baseline, WeirL = 10m',
                                                        color='black',
                                                        linestyle='--')
plt.legend()
plt.show()

In [ ]:
%%time
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Water Surface Elevation, lakes on (WeirL=10) and modified (WeirL=12m)',fontsize=24)

lakeout_weir12.sel(feature_id = 120053977).water_sfc_elev.plot(label='WeirL = 12m',
                                                        color='blue',
                                                        linestyle='-')
lakeout_lakes_on.sel(feature_id = 120053977).water_sfc_elev.plot(label='Baseline, WeirL = 10m',
                                                        color='black',
                                                        linestyle='--')
plt.legend()
plt.show()

### **On your own experiments**: 
In the above cells, modify the `inflow` variable selection in the plotting function above to `outflow`, then to `water_sfc_elev`.  Feel free to copy and paste the plotting cells so that each of the plots are saved. 

### Discussion
**Streamflow**: We expect that extending the weir length via the `WeirL` parameter causes the lake to discharge more water out of the weir when the lake level meets (exceeds) the weir elevation. This results an increase of streamflow downstream of the lake. 

**Lake outputs**: As expected, the inflow into the lake is the same in both simulations (baseline and modified WeirL). However, we find that the discharge is significantly different, due to more discharge out of the weir. From the lake level perspective, the baseline simulation shows that the lake continues to fill, and at a slower pace, the modified experiment with a longer WeirL keeps the lake level from reaching near its maximum.

# Next up - WRF-Hydro calibration
This concludes lesson 6. 

**IT IS BEST TO EITHER SHUTDOWN THIS LESSON OR CLOSE IT BEFORE PROCEEDING TO THE NEXT LESSON TO AVOID POSSIBLY EXCEEDING ALLOCATED MEMORY. Shutdown the lesson be either closing the browser tab for the lesson or selecting `Kernel -> Shut Down Kernel` in JupyterLab.**

© UCAR 2020